In [1]:
import re
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/steve/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/steve/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
f_shakespeare = open('../../data/shakespeare_cleaned.txt','r')
shakespeare = f_shakespeare.read()

f_obama = open('../../data/obama_cleaned.txt','r')
obama = f_obama.read()



In [70]:
n = 70 #amount of words in each data point
obama_words = obama.split(' ')
c_obama = [" ".join(obama_words[i:i+n]) for i in range(0, len(obama_words), n)]

print(np.average([len(x) for x in c_obama]))
print(len(c_obama))


384.4099099099099
444


In [71]:
n = 68 #amount of words/sentences in each data point
shake_words = shakespeare.split(' ')

#split based on n sentences - disabled for now
#c_shakespeare = [". ".join(shake_words[i:i+n]) for i in range(0, len(shake_words), n)]

#split based on n words
c_shakespeare = [" ".join(shake_words[i:i+n]) for i in range(0, len(shake_words), n)]

np.random.seed(59374)
np.random.shuffle(c_shakespeare)

print(np.average([len(x) for x in c_shakespeare]))
print(len(c_shakespeare))


370.10420193935664
12994


In [5]:
#naive bayes - big failure but good practice

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopset = list(set(stopwords.words('english')))

def word_feats(words):
    return dict([(word, True) for word in words.split() if word.lower() not in stopset])

l_shakespeare = [(word_feats(x), "shakespeare") for x in c_shakespeare]
l_obama = [(word_feats(x), "obama") for x in c_obama]

total_data = l_shakespeare[:444] + l_obama
np.random.seed(59374)
np.random.shuffle(total_data)

train_set, test_set = train_test_split(total_data)

classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)


1.0
Most Informative Features
                   shall = True           shakes : obama  =     35.4 : 1.0
                  people = True            obama : shakes =     27.5 : 1.0
                    also = True            obama : shakes =     22.3 : 1.0
                children = True            obama : shakes =     17.8 : 1.0
                    want = True            obama : shakes =     14.6 : 1.0


In [126]:

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import one_hot
from keras.utils import to_categorical
import operator
import random

class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles) if t not in string.punctuation and not any(char.isdigit() for char in t)]

def generate_vocab(data, vocab_size=None):
    """
    Generate vocabulary from a given string
    :param data: List of strings (should be denoised)
    :param vocab_size: Number of entries to save
    :return: Dictionary with words as the key and index as output
    """
    word_count = {}  # count of each word encountered
    word_index = {}  # vocabulary

    for word in data.split():
        if word not in word_count:
            word_count[word] = 1
        else:
            word_count[word] += 1
    if vocab_size is not None:
        ordered_words_by_index = sorted(word_count.items(), key=operator.itemgetter(1), reverse=True)
        ordered_words_by_index = ordered_words_by_index[:vocab_size]
        for word in ordered_words_by_index:
            word_index[word[0]] = len(word_index)
    else:
        for word in word_count:
            word_index[word] = len(word_index)
    return word_index

#total_data = [(x, "shakespeare") for x in c_shakespeare[:444]] + [(x, "obama") for x in c_obama]
q_shakespeare = [(x, "shakespeare") for x in c_shakespeare]
q_obama = [(x, "obama") for x in c_obama]
new_total = q_shakespeare[:444] + q_obama
np.random.seed(59374)
np.random.shuffle(new_total)

total_dict = {"text" : [x[0] for x in new_total], "author" : [x[1] for x in new_total]}

#corpus = ""
#for block in total_data:
#    corpus = corpus + block

#word_sequence = text_to_word_sequence(corpus)
#words = set(total_words)
#vocab_size = len(words)
#tokenizer = Tokenizer(num_words=2000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')

#final = one_hot(corpus, round(vocab_size*1.4))
#final = to_categorical(final)
#print(len(words))

#vocab = generate_vocab(corpus, vocab_size)
np.random.seed(59374)
np.random.shuffle(new_total)

#vectorizer = CountVectorizer(tokenizer=LemmaTokenizer())                         stop_words='english')
#X = vectorizer.fit_transform(total_data)
#print(X[:50])
#train_set, test_set = train_test_split(total_data)

#X = tokenizer.texts_to_sequences()
#X = pad_sequences(X)



In [2]:
from nltk.tokenize import sent_tokenize
import re
import pandas as pd
import numpy as np

def text_to_data(text, author):
    #remove newlines, numbers, some punctuation
    text = text.replace('\n', " ")
    text = re.sub(r'[0-9]+', '', text)
    sent_tokenize_list = sent_tokenize(text)    
    total_arr = [(x, author) for x in sent_tokenize_list]
    #total_dict = {"text" : [x for x in sent_tokenize_list], "author" : [author for x in sent_tokenize_list]}
    #total_dict = pd.DataFrame(data=total_dict)
    vocab_count = len(set(text.split(' ')))
    return total_arr, vocab_count

def shuffle_and_convert(dictionary):
    [[x, y] for x in dictionary['text'] for y in dictionary['author']]

total_word_count = 0
f = open('../../data/anthem.txt', 'r')
anthem_text, count = text_to_data(f.read(), "ayn rand")
total_word_count += count

f = open('../../data/mobydick.txt', 'r')
mobydick_text, count = text_to_data(f.read(), "herman melville")
total_word_count += count

f = open('../../data/alice.txt', 'r')
alice_text, count = text_to_data(f.read(), 'lewis carroll')
total_word_count += count

total_arr = anthem_text + mobydick_text + alice_text
np.random.seed(53894343)
np.random.shuffle(total_arr)

total_dict = {"text" : [x[0] for x in total_arr], "author" : [x[1] for x in total_arr]}
total_df = pd.DataFrame(total_dict)
print(total_df.describe())


                 author       text
count              3513       3513
unique                3       3479
top     herman melville  CHAPTER .
freq               1348         15


In [41]:
from keras.models import Sequential 
from keras.layers import Dense, Activation
from keras.preprocessing import text
from sklearn.preprocessing import LabelEncoder
from keras import utils
from keras.layers import Dense, Flatten, LSTM, Embedding, Dropout

def tokenize_input(train, test, vocab):
    # tokenize input text
    tokenizer = text.Tokenizer(num_words=vocab)
    tokenizer.fit_on_texts(train)
    # create input matrix
    return tokenizer.texts_to_matrix(train), \
           tokenizer.texts_to_matrix(test), \
           tokenizer

def encode_labels(train, test):
    encoder = LabelEncoder()
    encoder.fit(train)
    y_train = encoder.transform(train)
    num_classes = np.max(y_train) + 1
    return utils.to_categorical(y_train, num_classes), \
           utils.to_categorical(encoder.transform(test), num_classes), \
           encoder.classes_, num_classes

def split_dataset(dataset, split, input_field, label_field, ):
    train_size = int(len(dataset[input_field]) * split)
    return dataset[input_field][:train_size], dataset[input_field][train_size:], \
           dataset[label_field][:train_size], dataset[label_field][train_size:]

def init_data(data, vocab_size, data_split):
    train_input, test_input, train_label, test_label = split_dataset(total_dict, data_split, 'text', 'author')
    x_train, x_test, tokenizer = tokenize_input(train_input, test_input, vocab_size)
    y_train, y_test, text_labels, num_classes = encode_labels(train_label, test_label)
    return train_input, test_input, train_label, test_label, x_train, x_test, y_train, y_test, tokenizer, text_labels,\
           num_classes

def conv_x(xval, tokenizer):
    ret = tokenizer.texts_to_matrix(xval)
    return ret

def avgPrediction(predictionsArr):
    predictionAvgs = []
    average = 0;
    for x in range(len(predictionsArr[0])):
        for y in range(len(predictionsArr)):
            average += predictionsArr[y][x] / len(predictionsArr)
        predictionAvgs.append(average)
    # print it
            

train_input, test_input, train_label, test_label, x_train, x_test, y_train, y_test, tokenizer, text_labels, \
    num_classes = init_data(total_df, total_word_count, .7)

#pred_x = conv_x("Well, if I must, I must,’ the King said, with a melancholy air", tokenizer)
#pred_x = conv_x("Well, well, well! Stubb knows him best of all, and Stubb always says he’s queer; says nothing but that one sufficient little word queer; he’s queer, says Stubb; he’s queer—queer, queer; and keeps dinning it into Mr. Starbuck all the time—queer—sir—queer, queer, very queer. And here’s his leg! Yes, now that I think of it, here’s his bedfellow! has a stick of whale’s jaw-bone for a wife! And this is his leg; he’ll stand on this. What was that now about one leg standing in three places, and all three places standing in one hell—how was that? Oh! I don’t wonder he looked so scornful at me! I’m a sort of strange-thoughted sometimes, they say; but that’s only haphazard-like. Then, a short, little old body like me,", tokenizer)
pred_x = conv_x("Call me Ishmael. Some years ago—never mind how long precisely—having little or no money in my purse, and nothing particular to interest me on shore", tokenizer)

print(x_train.shape)
model = Sequential([
    Embedding(total_word_count, 32),
    LSTM(3),
    Dropout(.2),
    Dense(512, input_shape=(total_word_count,)),
    Dense(3, activation='sigmoid')
    #Dense(512),
    #Activation('sigmoid'), #TODO: change to sigmoid?
    #Dense(num_classes),
    #Activation('softmax')
])
model.compile(loss='categorical_crossentropy',optimizer ='adam',metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5, batch_size=32, verbose=1, validation_split=0.05)

pred = model.predict(pred_x)

#scores = model.evaluate(x_test, y_test, verbose=1)
#print(scores)

print(pred)
#print("Accuracy: %.2f%%" % (scores[1] * 100))
#model.fit(x_train,y_train,epochs=2000,batch_size=5,validation_split=0.05,verbose=0);
#print(model.summary())

(2459, 16765)
Train on 2336 samples, validate on 123 samples
Epoch 1/5
 160/2336 [=>............................] - ETA: 14:46 - loss: 1.0954 - acc: 0.4188

KeyboardInterrupt: 